# install modules and debendencies


In [1]:
!pip install transformers sentence-transformers faiss-cpu streamlit datasets pandas nltk
!pip install torch numpy scikit-learn matplotlib seaborn
!pip install huggingface_hub
!pip install streamlit
!pip install pyngrok
!pip install streamlit-chat
!pip install transformers torch accelerate bitsandbytes
!pip install sentencepiece protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.3 MB/s eta 0:00:00


# Import modules

In [2]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
import json
import random
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os
# Setup ngrok for public access
from pyngrok import ngrok
import time
from google.colab import userdata
import json
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    AutoModelForSequenceClassification
)
import warnings
warnings.filterwarnings("ignore")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Libraries imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: True


# import distilbert model and intialize it

In [3]:
# Load the pre-trained emotion detection model
model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
print(f"Loading emotion detection model: {model_name}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create emotion classification pipeline
emotion_classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

print("Emotion detection model loaded successfully!")
print(f"Model labels: {model.config.id2label}")

Loading emotion detection model: bhadresh-savani/distilbert-base-uncased-emotion


tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Device set to use cuda:0


Emotion detection model loaded successfully!
Model labels: {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}


# load empathetic-dialogues dataset from hugging-face and show it

In [4]:
# Load the empathetic dialogues dataset from Hugging Face
print("Loading empathetic dialogues dataset...")
dataset = load_dataset("bdotloh/empathetic-dialogues-contexts")

# Display dataset information
print(f"Dataset keys: {dataset.keys()}")
print(f"Train set size: {len(dataset['train'])}")
print(f"Dataset columns: {dataset['train'].column_names}")

# Display first few examples
print("\nFirst 5 examples:")
for i in range(5):
    example = dataset['train'][i]
    print(f"Example {i+1}:")
    print(f"  Situation: {example['situation']}")
    print(f"  Emotion: {example['emotion']}")
    print()

Loading empathetic dialogues dataset...


README.md:   0%|          | 0.00/540 [00:00<?, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/19209 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2756 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2542 [00:00<?, ? examples/s]

Dataset keys: dict_keys(['train', 'validation', 'test'])
Train set size: 19209
Dataset columns: ['Unnamed: 0', 'situation', 'emotion']

First 5 examples:
Example 1:
  Situation: I remember going to the fireworks with my best friend. There was a lot of people, but it only felt like us in the world.
  Emotion: sentimental

Example 2:
  Situation:  i used to scare for darkness
  Emotion: afraid

Example 3:
  Situation: I showed a guy how to run a good bead in welding class and he caught on quick.
  Emotion: proud

Example 4:
  Situation: I have always been loyal to my wife.
  Emotion: faithful

Example 5:
  Situation: A recent job interview that I had made me feel very anxious because I felt like I didn't come prepared.
  Emotion: terrified



# create the corpus json file

In [5]:
# Process the dataset and create corpus.json
print("Processing dataset and creating corpus.json...")

# Get all data
train_data = dataset['train']

# Count emotions
emotion_counts = {}
for item in train_data:
    emotion = item['emotion']
    emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

print("Emotion distribution in dataset:")
for emotion, count in sorted(emotion_counts.items()):
    print(f"  {emotion}: {count}")

# Sample ~1000 templates (or all if less than 1000)
sample_size = min(1000, len(train_data))
sampled_data = train_data.select(range(sample_size))

# Create corpus structure
corpus = {
    "happiness": [],
    "sadness": [],
    "anger": [],
    "neutral": []
}

# Map dataset emotions to our corpus categories
emotion_mapping = {
    'joy': 'happiness',
    'happiness': 'happiness',
    'excitement': 'happiness',
    'sadness': 'sadness',
    'sorrow': 'sadness',
    'grief': 'sadness',
    'anger': 'anger',
    'rage': 'anger',
    'fury': 'anger',
    'neutral': 'neutral',
    'calm': 'neutral',
    'peaceful': 'neutral'
}

# Process each item
for item in sampled_data:
    situation = item['situation']
    emotion = item['emotion'].lower()

    # Map to our categories
    mapped_emotion = emotion_mapping.get(emotion, 'neutral')

    # Create response template
    response_template = {
        "situation": situation,
        "emotion": emotion,
        "response_template": f"I understand you're feeling {emotion} about {situation[:50]}...",
        "context": situation
    }

    corpus[mapped_emotion].append(response_template)

# Print corpus statistics
print(f"\nCorpus created with {sample_size} samples:")
for emotion, responses in corpus.items():
    print(f"  {emotion}: {len(responses)} responses")

# Save to JSON file
with open('corpus.json', 'w', encoding='utf-8') as f:
    json.dump(corpus, f, indent=2, ensure_ascii=False)

print(f"\nCorpus saved to corpus.json")
print(f"Total responses: {sum(len(responses) for responses in corpus.values())}")

Processing dataset and creating corpus.json...
Emotion distribution in dataset:
  afraid: 619
  angry: 677
  annoyed: 657
  anticipating: 597
  anxious: 611
  apprehensive: 460
  ashamed: 485
  caring: 504
  confident: 610
  content: 568
  devastated: 553
  disappointed: 593
  disgusted: 611
  embarrassed: 560
  excited: 731
  faithful: 372
  furious: 593
  grateful: 637
  guilty: 611
  hopeful: 615
  impressed: 612
  jealous: 579
  joyful: 596
  lonely: 633
  nostalgic: 595
  prepared: 584
  proud: 666
  sad: 660
  sentimental: 512
  surprised: 997
  terrified: 614
  trusting: 497

Corpus created with 1000 samples:
  happiness: 0 responses
  sadness: 0 responses
  anger: 0 responses
  neutral: 1000 responses

Corpus saved to corpus.json
Total responses: 1000


# test the model with some samples to ensure it fit with the requirements

In [6]:
# Test the emotion model with sample messages
print("Testing emotion detection model...")

# Sample test messages covering different emotions
test_messages = [
    "I'm so excited about my new job!",
    "I can't believe my best friend moved away.",
    "This traffic is making me furious!",
    "I had a peaceful walk in the park today.",
    "I'm feeling really sad about the news.",
    "What an amazing surprise party!",
    "I'm so angry at my boss right now.",
    "The weather is nice and calm today.",
    "I'm thrilled about my vacation next week.",
    "I feel so lonely without my family."
]

print("Testing 10 sample messages:")
print("=" * 60)

for i, message in enumerate(test_messages, 1):
    # Get emotion predictions
    predictions = emotion_classifier(message)

    # Get the top prediction
    top_prediction = max(predictions[0], key=lambda x: x['score'])

    print(f"\nTest {i}:")
    print(f"Message: '{message}'")
    print(f"Predicted emotion: {top_prediction['label']} (confidence: {top_prediction['score']:.3f})")

    # Show all predictions
    print("All predictions:")
    for pred in sorted(predictions[0], key=lambda x: x['score'], reverse=True):
        print(f"  {pred['label']}: {pred['score']:.3f}")

Testing emotion detection model...
Testing 10 sample messages:

Test 1:
Message: 'I'm so excited about my new job!'
Predicted emotion: joy (confidence: 0.999)
All predictions:
  joy: 0.999
  love: 0.000
  surprise: 0.000
  fear: 0.000
  anger: 0.000
  sadness: 0.000

Test 2:
Message: 'I can't believe my best friend moved away.'
Predicted emotion: joy (confidence: 0.544)
All predictions:
  joy: 0.544
  sadness: 0.343
  fear: 0.074
  anger: 0.030
  surprise: 0.006
  love: 0.003

Test 3:
Message: 'This traffic is making me furious!'
Predicted emotion: anger (confidence: 0.997)
All predictions:
  anger: 0.997
  fear: 0.001
  sadness: 0.000
  joy: 0.000
  love: 0.000
  surprise: 0.000

Test 4:
Message: 'I had a peaceful walk in the park today.'
Predicted emotion: joy (confidence: 0.999)
All predictions:
  joy: 0.999
  love: 0.000
  sadness: 0.000
  anger: 0.000
  fear: 0.000
  surprise: 0.000

Test 5:
Message: 'I'm feeling really sad about the news.'
Predicted emotion: sadness (confidence: 

# Load the sentence transfromer model to make the embeddings

In [7]:
# Load the sentence transformer model
print("Loading Sentence-Transformers model...")
model_name = "all-MiniLM-L6-v2"
sentence_model = SentenceTransformer(model_name)

print(f"Model loaded: {model_name}")
print(f"Model max sequence length: {sentence_model.max_seq_length}")
print(f"Embedding dimension: {sentence_model.get_sentence_embedding_dimension()}")

Loading Sentence-Transformers model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded: all-MiniLM-L6-v2
Model max sequence length: 256
Embedding dimension: 384


# processing the data

In [8]:
# Load the corpus data
print("Loading corpus data...")
with open('corpus.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)

# Flatten the corpus into a single list with emotion labels
all_responses = []
emotion_labels = []

for emotion, responses in corpus.items():
    for response in responses:
        # Create a combined text for embedding (situation + response template)
        combined_text = f"{response['situation']} {response['response_template']}"

        all_responses.append({
            'text': combined_text,
            'situation': response['situation'],
            'response_template': response['response_template'],
            'emotion': emotion,
            'original_emotion': response['emotion']
        })
        emotion_labels.append(emotion)

print(f"Total responses loaded: {len(all_responses)}")
print(f"Emotion distribution:")
for emotion in set(emotion_labels):
    count = emotion_labels.count(emotion)
    print(f"  {emotion}: {count}")

Loading corpus data...
Total responses loaded: 1000
Emotion distribution:
  neutral: 1000


# generate the embeddings

In [9]:
# Generate embeddings for all responses
print("Generating embeddings...")

# Extract text for embedding
texts_to_embed = [response['text'] for response in all_responses]

# Generate embeddings
print(f"Processing {len(texts_to_embed)} texts...")
embeddings = sentence_model.encode(texts_to_embed, show_progress_bar=True)

print(f"Embeddings shape: {embeddings.shape}")
print(f"Embedding dimension: {embeddings.shape[1]}")

# Verify embeddings
print(f"Sample embedding (first 5 values): {embeddings[0][:5]}")
print(f"Embedding norm: {np.linalg.norm(embeddings[0]):.4f}")

Generating embeddings...
Processing 1000 texts...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Embeddings shape: (1000, 384)
Embedding dimension: 384
Sample embedding (first 5 values): [ 0.05622683  0.04485424  0.07846128 -0.01251352  0.04654704]
Embedding norm: 1.0000


# intizalize the FAISS vector database and store the embeddings

In [10]:
# Create FAISS index for efficient similarity search
print("Setting up FAISS index...")

# Get embedding dimension
embedding_dim = embeddings.shape[1]

# Create FAISS index (using L2 distance)
index = faiss.IndexFlatIP(embedding_dim)  # Inner Product for cosine similarity

# Normalize embeddings for cosine similarity
embeddings_normalized = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Add embeddings to index
index.add(embeddings_normalized.astype('float32'))

print(f"FAISS index created with {index.ntotal} vectors")
print(f"Index dimension: {index.d}")

# Save the index for future use
faiss.write_index(index, "corpus_embeddings.index")
print("FAISS index saved to 'corpus_embeddings.index'")

Setting up FAISS index...
FAISS index created with 1000 vectors
Index dimension: 384
FAISS index saved to 'corpus_embeddings.index'


# intialize the retriver

In [11]:
class EmotionAwareRetriever:
    def __init__(self, sentence_model, faiss_index, all_responses, emotion_classifier):
        self.sentence_model = sentence_model
        self.faiss_index = faiss_index
        self.all_responses = all_responses
        self.emotion_classifier = emotion_classifier

    def detect_emotion(self, user_input):
        """Detect emotion from user input"""
        predictions = self.emotion_classifier(user_input)
        top_prediction = max(predictions[0], key=lambda x: x['score'])
        return top_prediction['label'], top_prediction['score']

    def map_emotion_to_category(self, detected_emotion):
        """Map detected emotion to our corpus categories"""
        emotion_mapping = {
            'joy': 'happiness',
            'happiness': 'happiness',
            'excitement': 'happiness',
            'love': 'happiness',
            'surprise': 'happiness',
            'sadness': 'sadness',
            'sorrow': 'sadness',
            'grief': 'sadness',
            'anger': 'anger',
            'rage': 'anger',
            'fury': 'anger',
            'fear': 'neutral',
            'neutral': 'neutral',
            'calm': 'neutral',
            'peaceful': 'neutral'
        }
        return emotion_mapping.get(detected_emotion.lower(), 'neutral')

    def retrieve_responses(self, user_input, top_k=3, emotion_weight=0.7):
        """
        Retrieve top-k responses based on user input and detected emotion

        Args:
            user_input: User's message
            top_k: Number of responses to retrieve
            emotion_weight: Weight for emotion matching (0.0 to 1.0)
        """
        # Detect emotion
        detected_emotion, confidence = self.detect_emotion(user_input)
        mapped_emotion = self.map_emotion_to_category(detected_emotion)

        print(f"Detected emotion: {detected_emotion} (confidence: {confidence:.3f})")
        print(f"Mapped to category: {mapped_emotion}")

        # Generate embedding for user input
        user_embedding = self.sentence_model.encode([user_input])
        user_embedding_normalized = user_embedding / np.linalg.norm(user_embedding, axis=1, keepdims=True)

        # Search for similar responses
        scores, indices = self.faiss_index.search(user_embedding_normalized.astype('float32'), top_k * 3)  # Get more candidates

        # Filter and rank by emotion alignment
        candidates = []
        for score, idx in zip(scores[0], indices[0]):
            response = self.all_responses[idx]
            emotion_match = 1.0 if response['emotion'] == mapped_emotion else 0.0

            # Combined score: semantic similarity + emotion alignment
            combined_score = (1 - emotion_weight) * score + emotion_weight * emotion_match

            candidates.append({
                'response': response,
                'semantic_score': float(score),
                'emotion_match': emotion_match,
                'combined_score': float(combined_score),
                'emotion_category': response['emotion']
            })

        # Sort by combined score and return top-k
        candidates.sort(key=lambda x: x['combined_score'], reverse=True)

        return {
            'detected_emotion': detected_emotion,
            'mapped_emotion': mapped_emotion,
            'confidence': confidence,
            'retrieved_responses': candidates[:top_k]
        }

# Initialize the retriever
retriever = EmotionAwareRetriever(sentence_model, index, all_responses, emotion_classifier)
print("EmotionAwareRetriever initialized successfully!")

EmotionAwareRetriever initialized successfully!


# test the retriver to check it's accuracy


In [12]:
# Test the retrieval system with sample queries
print("Testing Emotion-Aware Retrieval System")
print("=" * 50)

test_queries = [
    "I'm so excited about my new job!",
    "I can't believe my best friend moved away.",
    "This traffic is making me furious!",
    "I had a peaceful walk in the park today.",
    "I'm feeling really sad about the news."
]

for i, query in enumerate(test_queries, 1):
    print(f"\nTest Query {i}: '{query}'")
    print("-" * 40)

    # Retrieve responses
    results = retriever.retrieve_responses(query, top_k=3)

    print(f"Detected: {results['detected_emotion']} (confidence: {results['confidence']:.3f})")
    print(f"Mapped to: {results['mapped_emotion']}")
    print("\nTop 3 Retrieved Responses:")

    for j, candidate in enumerate(results['retrieved_responses'], 1):
        response = candidate['response']
        print(f"\n  {j}. Emotion: {candidate['emotion_category']}")
        print(f"     Semantic Score: {candidate['semantic_score']:.3f}")
        print(f"     Emotion Match: {candidate['emotion_match']}")
        print(f"     Combined Score: {candidate['combined_score']:.3f}")
        print(f"     Situation: {response['situation'][:100]}...")
        print(f"     Response: {response['response_template']}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Testing Emotion-Aware Retrieval System

Test Query 1: 'I'm so excited about my new job!'
----------------------------------------
Detected emotion: joy (confidence: 0.999)
Mapped to category: happiness
Detected: joy (confidence: 0.999)
Mapped to: happiness

Top 3 Retrieved Responses:

  1. Emotion: neutral
     Semantic Score: 0.668
     Emotion Match: 0.0
     Combined Score: 0.200
     Situation: I'm so happy I found a new job!...
     Response: I understand you're feeling joyful about I'm so happy I found a new job!...

  2. Emotion: neutral
     Semantic Score: 0.596
     Emotion Match: 0.0
     Combined Score: 0.179
     Situation: After 4 months of going on interviews.  Today I finally got a job!...
     Response: I understand you're feeling joyful about After 4 months of going on interviews.  Today I fi...

  3. Emotion: neutral
     Semantic Score: 0.573
     Emotion Match: 0.0
     Combined Score: 0.172
     Situation: I was really nervous when starting a new job a few months 

# more comprehenseve testing for the retriver

In [13]:
# Test with more diverse queries and analyze performance
print("Advanced Retrieval Testing")
print("=" * 40)

# Test queries with different emotional contexts
advanced_queries = [
    ("I got promoted at work!", "happiness"),
    ("My pet passed away yesterday", "sadness"),
    ("I'm so mad at my roommate", "anger"),
    ("Just had a relaxing weekend", "neutral"),
    ("I'm terrified of the presentation tomorrow", "fear"),
    ("My partner surprised me with flowers", "happiness"),
    ("I feel so alone in this city", "sadness"),
    ("This customer service is terrible!", "anger"),
    ("The weather is perfect today", "neutral"),
    ("I'm anxious about the job interview", "fear")
]

print("Testing emotion-specific retrieval:")
print("-" * 40)

for query, expected_emotion in advanced_queries:
    print(f"\nQuery: '{query}'")
    print(f"Expected emotion category: {expected_emotion}")

    results = retriever.retrieve_responses(query, top_k=3, emotion_weight=0.8)

    # Check if top response matches expected emotion
    top_response = results['retrieved_responses'][0]
    emotion_match = top_response['emotion_category'] == expected_emotion

    print(f"Detected: {results['detected_emotion']} → {results['mapped_emotion']}")
    print(f"Top response emotion: {top_response['emotion_category']}")
    print(f"Emotion match: {'✓' if emotion_match else '✗'}")
    print(f"Combined score: {top_response['combined_score']:.3f}")

Advanced Retrieval Testing
Testing emotion-specific retrieval:
----------------------------------------

Query: 'I got promoted at work!'
Expected emotion category: happiness
Detected emotion: joy (confidence: 0.971)
Mapped to category: happiness
Detected: joy → happiness
Top response emotion: neutral
Emotion match: ✗
Combined score: 0.138

Query: 'My pet passed away yesterday'
Expected emotion category: sadness
Detected emotion: sadness (confidence: 0.991)
Mapped to category: sadness
Detected: sadness → sadness
Top response emotion: neutral
Emotion match: ✗
Combined score: 0.143

Query: 'I'm so mad at my roommate'
Expected emotion category: anger
Detected emotion: anger (confidence: 0.998)
Mapped to category: anger
Detected: anger → anger
Top response emotion: neutral
Emotion match: ✗
Combined score: 0.104

Query: 'Just had a relaxing weekend'
Expected emotion category: neutral
Detected emotion: joy (confidence: 0.999)
Mapped to category: happiness
Detected: joy → happiness
Top respon

# save the RAG system

In [14]:
# Save the complete retrieval system for future use
print("Saving retrieval system...")

# Save the sentence model (it will be cached automatically)
print("Sentence model will be cached automatically")

# Save the FAISS index (already saved)
print("FAISS index saved as 'corpus_embeddings.index'")

# Save the response data
with open('response_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_responses, f, indent=2, ensure_ascii=False)

print("Response data saved as 'response_data.json'")

# Save the retriever configuration
retriever_config = {
    'model_name': model_name,
    'embedding_dimension': embeddings.shape[1],
    'total_responses': len(all_responses),
    'emotion_categories': list(set(emotion_labels)),
    'faiss_index_file': 'corpus_embeddings.index',
    'response_data_file': 'response_data.json'
}

with open('retriever_config.json', 'w', encoding='utf-8') as f:
    json.dump(retriever_config, f, indent=2)

print("Retriever configuration saved as 'retriever_config.json'")
print("\nRetrieval system setup complete!")
print(f"Total responses indexed: {len(all_responses)}")
print(f"Embedding dimension: {embeddings.shape[1]}")
print(f"FAISS index size: {index.ntotal} vectors")

Saving retrieval system...
Sentence model will be cached automatically
FAISS index saved as 'corpus_embeddings.index'
Response data saved as 'response_data.json'
Retriever configuration saved as 'retriever_config.json'

Retrieval system setup complete!
Total responses indexed: 1000
Embedding dimension: 384
FAISS index size: 1000 vectors


In [15]:
# Analyze retrieval performance
print("Retrieval System Performance Analysis")
print("=" * 40)

# Test retrieval speed
import time

test_query = "I'm feeling really sad about losing my job"
num_tests = 10

print(f"Testing retrieval speed with '{test_query}'")
print(f"Running {num_tests} iterations...")

start_time = time.time()
for _ in range(num_tests):
    results = retriever.retrieve_responses(test_query, top_k=3)
end_time = time.time()

avg_time = (end_time - start_time) / num_tests
print(f"Average retrieval time: {avg_time:.4f} seconds")
print(f"Retrievals per second: {1/avg_time:.1f}")

# Memory usage
import sys
print(f"\nMemory usage:")
print(f"Embeddings size: {embeddings.nbytes / (1024*1024):.2f} MB")
print(f"FAISS index size: {sys.getsizeof(index) / (1024*1024):.2f} MB")
print(f"Response data size: {sys.getsizeof(all_responses) / (1024*1024):.2f} MB")

print("\n✅ RAG System Setup Complete!")
print("The system is ready for emotion-aware response retrieval.")

Retrieval System Performance Analysis
Testing retrieval speed with 'I'm feeling really sad about losing my job'
Running 10 iterations...
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Detected emotion: sadness (confidence: 0.999)
Mapped to category: sadness
Average retrieval time: 0.0209 seconds
Retrievals per second: 47.9

Memory usage:
Embeddings size: 1.46 MB
FAISS index size

# load DialoGPT to test with it

In [16]:
# Load a smaller, efficient model that can run in Colab
print("Loading pre-trained language model...")

# Using a smaller, efficient model that works well in Colab
model_name = "microsoft/DialoGPT-medium"  # Good for conversational AI
# Alternative options:
# "distilgpt2" - Very fast, smaller
# "microsoft/DialoGPT-small" - Smaller version
# "facebook/blenderbot-400M-distill" - Good for empathetic responses

try:
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )

    # Add padding token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print(f"✅ Model loaded successfully: {model_name}")
    print(f"Model parameters: {model.num_parameters():,}")

except Exception as e:
    print(f"❌ Error loading {model_name}: {e}")
    print("Falling back to DistilGPT2...")

    # Fallback to a smaller model
    model_name = "distilgpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print(f"✅ Fallback model loaded: {model_name}")

Loading pre-trained language model...


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded successfully: microsoft/DialoGPT-medium
Model parameters: 354,823,168


# Define the few shot prompts

In [17]:
# Define few-shot prompt templates for different emotions
FEW_SHOT_TEMPLATES = {
    'happiness': {
        'system_prompt': "You are an empathetic AI assistant. The user is feeling happy/excited. Respond with genuine enthusiasm and support.",
        'examples': [
            {
                'user': "I'm so excited about my new job!",
                'bot': "That's wonderful! I can really feel your excitement about this new opportunity. New jobs often bring fresh challenges and growth - I'm genuinely happy for you! What aspect of the new role are you most looking forward to?"
            },
            {
                'user': "I just got accepted to my dream university!",
                'bot': "Congratulations! That's absolutely fantastic news! I can sense how thrilled and proud you must be feeling right now. Getting into your dream university is such a significant achievement - you should be really proud of yourself!"
            }
        ]
    },
    'sadness': {
        'system_prompt': "You are an empathetic AI assistant. The user is feeling sad or grieving. Respond with compassion, understanding, and gentle support.",
        'examples': [
            {
                'user': "I'm feeling really sad about losing my job.",
                'bot': "I'm so sorry to hear about your job loss. That must be incredibly difficult and overwhelming for you right now. It's completely understandable to feel sad about this - job loss affects so many aspects of life. You're not alone in this, and it's okay to take time to process these feelings."
            },
            {
                'user': "My best friend moved away and I feel so lonely.",
                'bot': "I can really understand how lonely and sad you must be feeling right now. Losing that close connection when a friend moves away is genuinely heartbreaking. It's natural to grieve this change in your relationship. Your feelings are completely valid."
            }
        ]
    },
    'anger': {
        'system_prompt': "You are an empathetic AI assistant. The user is feeling angry or frustrated. Respond with understanding, validation, and help them process their feelings constructively.",
        'examples': [
            {
                'user': "I'm so angry at my boss for treating me unfairly!",
                'bot': "I can really understand why you're feeling angry about this situation. Being treated unfairly at work is incredibly frustrating and it's completely valid to feel this way. Your anger is telling you that something important to you - fairness and respect - has been violated. Would you like to talk about what happened?"
            },
            {
                'user': "This traffic is making me furious!",
                'bot': "I totally get why you're feeling furious right now - traffic can be incredibly frustrating, especially when you have somewhere important to be! It's so annoying when things are out of your control like this. Your frustration is completely understandable."
            }
        ]
    },
    'neutral': {
        'system_prompt': "You are an empathetic AI assistant. The user seems to be in a neutral or calm state. Respond with gentle engagement and support.",
        'examples': [
            {
                'user': "I had a peaceful walk in the park today.",
                'bot': "That sounds really lovely and refreshing! Taking peaceful walks can be so grounding and good for both your mind and body. I'm glad you were able to enjoy that quiet time in nature. How did it make you feel?"
            },
            {
                'user': "The weather is nice and calm today.",
                'bot': "That sounds wonderful! Nice, calm weather can really lift your spirits and make everything feel a bit more pleasant. I hope you're able to enjoy it and that it brings you some peace today."
            }
        ]
    }
}

print("Few-shot prompt templates created successfully!")

Few-shot prompt templates created successfully!


# initialize AI Pipeline with pre-trained model

In [18]:
# Fix the pipeline issues
class FixedEmpathyAIPipeline:
    def __init__(self, emotion_classifier, retriever, few_shot_templates, language_model, tokenizer):
        self.emotion_classifier = emotion_classifier
        self.retriever = retriever
        self.few_shot_templates = few_shot_templates
        self.language_model = language_model
        self.tokenizer = tokenizer
        self.device = next(language_model.parameters()).device

    def generate_response(self, user_message, use_rag=True, use_few_shot=True, max_length=100):
        """Fixed pipeline with better response generation"""
        print(f"\n Processing: '{user_message}'")
        print("-" * 50)

        # Step 1: Detect emotion
        print("Step 1: Detecting emotion...")
        predictions = self.emotion_classifier(user_message)
        top_prediction = max(predictions[0], key=lambda x: x['score'])
        detected_emotion = top_prediction['label']
        confidence = top_prediction['score']

        print(f"   Detected: {detected_emotion} (confidence: {confidence:.3f})")

        # Step 2: Map emotion to category (FIXED)
        emotion_mapping = {
            'joy': 'happiness', 'happiness': 'happiness', 'excitement': 'happiness',
            'love': 'happiness', 'surprise': 'happiness',
            'sadness': 'sadness', 'sorrow': 'sadness', 'grief': 'sadness',
            'anger': 'anger', 'rage': 'anger', 'fury': 'anger',
            'fear': 'fear',  # FIXED: fear should map to fear, not neutral
            'neutral': 'neutral', 'calm': 'neutral', 'peaceful': 'neutral'
        }
        emotion_category = emotion_mapping.get(detected_emotion.lower(), 'neutral')
        print(f"   Mapped to category: {emotion_category}")

        # Step 3: Retrieve relevant templates (if using RAG)
        if use_rag:
            print("Step 2: Retrieving relevant templates...")
            rag_results = self.retriever.retrieve_responses(user_message, top_k=3, emotion_weight=0.8)
            templates = rag_results['retrieved_responses']
            print(f"   Retrieved {len(templates)} templates")
        else:
            templates = []

        # Step 4: Generate response using improved method
        print("Step 3: Generating response with pre-trained model...")
        response = self._generate_improved_response(user_message, detected_emotion, emotion_category, templates, max_length)

        # Step 5: Add disclaimer
        print("Step 4: Adding disclaimer...")
        final_response = self._add_disclaimer(response)

        return {
            'user_message': user_message,
            'detected_emotion': detected_emotion,
            'emotion_category': emotion_category,
            'confidence': confidence,
            'templates_used': templates,
            'response': final_response
        }

    def _generate_improved_response(self, user_message, detected_emotion, emotion_category, templates, max_length=100):
        """Generate response using an improved approach"""

        # Create a more focused prompt
        prompt = self._create_focused_prompt(user_message, detected_emotion, emotion_category)

        # Tokenize the prompt and move to correct device
        inputs = self.tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=400)
        inputs = inputs.to(self.device)

        # Generate response with better parameters
        with torch.no_grad():
            outputs = self.language_model.generate(
                inputs,
                max_length=inputs.shape[1] + max_length,
                num_return_sequences=1,
                temperature=0.8,
                do_sample=True,
                top_p=0.9,
                top_k=40,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                no_repeat_ngram_size=3,
                attention_mask=torch.ones_like(inputs),
                repetition_penalty=1.1
            )

        # Decode the response
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the bot's response
        bot_response = self._extract_bot_response(full_response, prompt)

        # If the response is too generic or doesn't make sense, use a fallback
        if len(bot_response) < 20 or self._is_generic_response(bot_response):
            bot_response = self._generate_fallback_response(user_message, detected_emotion, emotion_category, templates)

        return bot_response

    def _create_focused_prompt(self, user_message, detected_emotion, emotion_category):
        """Create a more focused prompt"""

        # Get appropriate examples based on emotion
        if emotion_category == 'happiness':
            examples = [
                "User: I'm so excited about my new job!\nAssistant: That's wonderful! I can really feel your excitement about this new opportunity. What aspect of the new role are you most looking forward to?",
                "User: My partner surprised me with flowers!\nAssistant: How sweet and thoughtful! I can sense how happy and loved you must be feeling right now. That's such a lovely gesture!"
            ]
        elif emotion_category == 'sadness':
            examples = [
                "User: I'm feeling really sad about the news.\nAssistant: I'm so sorry you're feeling this way. It's completely understandable to feel sad about this. I'm here to listen and support you.",
                "User: I feel so alone in this city.\nAssistant: I can really understand how lonely and isolated you must be feeling right now. Your feelings are completely valid, and I'm here to listen."
            ]
        elif emotion_category == 'anger':
            examples = [
                "User: This traffic is making me furious!\nAssistant: I totally get why you're feeling furious right now - traffic can be incredibly frustrating! Your frustration is completely understandable.",
                "User: I'm so angry at my boss!\nAssistant: I can really understand why you're feeling angry about this situation. It's completely valid to feel this way. Would you like to talk about what happened?"
            ]
        elif emotion_category == 'fear':
            examples = [
                "User: I'm terrified of the presentation tomorrow.\nAssistant: I can really understand why you're feeling anxious about this presentation. It's completely normal to feel nervous about important presentations. What specifically is worrying you about it?",
                "User: I'm scared about the job interview.\nAssistant: I understand why you're feeling scared about the interview. It's natural to feel anxious about something so important. What's making you most nervous about it?"
            ]
        else:  # neutral
            examples = [
                "User: I had a peaceful walk in the park today.\nAssistant: That sounds really lovely and refreshing! Taking peaceful walks can be so grounding. How did it make you feel?",
                "User: The weather is nice and calm today.\nAssistant: That sounds wonderful! Nice, calm weather can really lift your spirits. I hope you're able to enjoy it."
            ]

        # Create the prompt
        prompt_parts = [
            "You are an empathetic AI assistant. Respond with understanding and compassion.",
            "",
            "Examples:",
            ""
        ]

        for example in examples:
            prompt_parts.append(example)
            prompt_parts.append("")

        prompt_parts.extend([
            f"Now respond empathetically to this user message:",
            f"User: {user_message}",
            "Assistant:"
        ])

        return "\n".join(prompt_parts)

    def _extract_bot_response(self, full_response, prompt):
        """Extract only the bot's response from the generated text"""
        # Find where the prompt ends and the response begins
        prompt_end = full_response.find("Assistant:")
        if prompt_end != -1:
            response = full_response[prompt_end + len("Assistant:"):].strip()
        else:
            response = full_response[len(prompt):].strip()

        # Clean up the response
        response = response.split('\n')[0]  # Take only the first line
        response = response.strip()

        return response

    def _is_generic_response(self, response):
        """Check if the response is too generic"""
        generic_phrases = [
            "That's wonderful! I can really feel your excitement about this new opportunity",
            "I'm so sorry to hear about your job loss",
            "I can really understand why you're feeling angry about this situation"
        ]
        return any(phrase in response for phrase in generic_phrases)

    def _generate_fallback_response(self, user_message, detected_emotion, emotion_category, templates):
        """Generate a fallback response if the LLM response is poor"""

        # Use templates if available
        if templates:
            base_template = templates[0]['response']['response_template']
        else:
            base_template = f"I understand you're feeling {detected_emotion}."

        # Create contextual responses
        if emotion_category == 'happiness':
            return f"I can really feel your joy and excitement! {base_template} That's wonderful to hear. What's making you feel this way?"
        elif emotion_category == 'sadness':
            return f"I'm so sorry you're feeling this way. {base_template} Your feelings are completely valid and I'm here to listen. Would you like to talk about what's on your mind?"
        elif emotion_category == 'anger':
            return f"I can understand why you're feeling frustrated. {base_template} It's okay to feel angry about this. What's been frustrating you lately?"
        elif emotion_category == 'fear':
            return f"I can really understand why you're feeling anxious about this. {base_template} It's completely normal to feel nervous. What's worrying you most about it?"
        else:
            return f"I'm here to listen and support you. {base_template} How are you feeling about everything?"

    def _add_disclaimer(self, response):
        """Add disclaimer to the response"""
        disclaimer = "\n\n⚠️ *Disclaimer: I'm an AI assistant designed to provide empathetic responses. I'm not a substitute for professional mental health care. If you're experiencing severe emotional distress, please consider reaching out to a mental health professional.*"
        return response + disclaimer

# Initialize the fixed pipeline
fixed_pipeline = FixedEmpathyAIPipeline(
    emotion_classifier,
    retriever,
    FEW_SHOT_TEMPLATES,
    model,
    tokenizer
)
print("✅ Fixed Empathy AI Pipeline initialized!")

✅ Fixed Empathy AI Pipeline initialized!


# Test the Full pipeline

In [19]:
# Test the fixed pipeline
print("Testing Fixed Empathy AI Pipeline")
print("=" * 60)

test_messages = [
    "I'm so excited about my new job!",
    "I can't believe my best friend moved away.",
    "This traffic is making me furious!",
    "I had a peaceful walk in the park today.",
    "I'm feeling really sad about the news.",
    "I'm terrified of the presentation tomorrow",
    "My partner surprised me with flowers",
    "I feel so alone in this city"
]

for i, message in enumerate(test_messages, 1):
    print(f"\n{'='*60}")
    print(f"TEST {i}")
    print(f"{'='*60}")

    result = fixed_pipeline.generate_response(message, use_rag=True, use_few_shot=True)

    print(f"\n🤖 BOT RESPONSE:")
    print(f"{result['response']}")
    print(f"\n📊 ANALYSIS:")
    print(f"   Emotion: {result['detected_emotion']} (confidence: {result['confidence']:.3f})")
    print(f"   Category: {result['emotion_category']}")
    print(f"   Templates used: {len(result['templates_used'])}")

Testing Fixed Empathy AI Pipeline

TEST 1

 Processing: 'I'm so excited about my new job!'
--------------------------------------------------
Step 1: Detecting emotion...
   Detected: joy (confidence: 0.999)
   Mapped to category: happiness
Step 2: Retrieving relevant templates...
Detected emotion: joy (confidence: 0.999)
Mapped to category: happiness
   Retrieved 3 templates
Step 3: Generating response with pre-trained model...
Step 4: Adding disclaimer...

🤖 BOT RESPONSE:
I can really feel your joy and excitement! I understand you're feeling joyful about I'm so happy I found a new job!... That's wonderful to hear. What's making you feel this way?

⚠️ *Disclaimer: I'm an AI assistant designed to provide empathetic responses. I'm not a substitute for professional mental health care. If you're experiencing severe emotional distress, please consider reaching out to a mental health professional.*

📊 ANALYSIS:
   Emotion: joy (confidence: 0.999)
   Category: happiness
   Templates used: 3


# conversational mode

In [20]:
# Interactive chat with the fixed pipeline
def fixed_interactive_chat():
    """Interactive chat using the fixed pipeline"""
    print("\n" + "="*70)
    print("🤖 FIXED EMPATHY AI CHAT - Real Language Model")
    print("="*70)
    print("Type your messages below. Type 'quit', 'exit', or 'bye' to end the chat.")
    print("Type 'help' for available commands.")
    print("-"*70)

    chat_history = []

    while True:
        try:
            # Get user input
            user_input = input("\n👤 You: ").strip()

            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', 'q']:
                print("\n�� Bot: Thank you for chatting with me! Take care and remember that your feelings are valid. Goodbye! 👋")
                break

            # Check for help command
            if user_input.lower() == 'help':
                print("\n Available commands:")
                print("   - Type any message to get an empathetic response")
                print("   - 'quit', 'exit', 'bye', or 'q' to end the chat")
                print("   - 'history' to see chat history")
                print("   - 'clear' to clear chat history")
                continue

            # Check for history command
            if user_input.lower() == 'history':
                if chat_history:
                    print("\n📜 Chat History:")
                    for i, entry in enumerate(chat_history, 1):
                        print(f"   {i}. You: {entry['user']}")
                        print(f"      Bot: {entry['bot'][:100]}...")
                else:
                    print("\n📜 No chat history yet.")
                continue

            # Check for clear command
            if user_input.lower() == 'clear':
                chat_history = []
                print("\n🗑️ Chat history cleared.")
                continue

            # Skip empty messages
            if not user_input:
                continue

            # Generate response
            print("\n🤖 Bot: Thinking...")
            result = fixed_pipeline.generate_response(user_input, use_rag=True, use_few_shot=True)

            # Store in history
            chat_history.append({
                'user': user_input,
                'bot': result['response'],
                'emotion': result['detected_emotion'],
                'confidence': result['confidence']
            })

            # Display response
            print(f"\n🤖 Bot: {result['response']}")

        except KeyboardInterrupt:
            print("\n\n🤖 Bot: Chat interrupted. Goodbye! 👋")
            break
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
            print("Please try again or type 'quit' to exit.")

# Start fixed interactive chat
print("Starting fixed interactive chat...")
fixed_interactive_chat()

Starting fixed interactive chat...

🤖 FIXED EMPATHY AI CHAT - Real Language Model
Type your messages below. Type 'quit', 'exit', or 'bye' to end the chat.
Type 'help' for available commands.
----------------------------------------------------------------------


🤖 Bot: Chat interrupted. Goodbye! 👋


# create the streamlit app code

In [21]:
# Create the file line by line with response cleaning
lines = [
    "import streamlit as st",
    "import json",
    "import torch",
    "import numpy as np",
    "from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline",
    "from sentence_transformers import SentenceTransformer",
    "import faiss",
    "import pickle",
    "import os",
    "from datetime import datetime",
    "import random",
    "import time",
    "import re",
    "",
    "# Page configuration",
    "st.set_page_config(",
    "    page_title='Empathy AI Chatbot - Phi-2 + RAG',",
    "    page_icon='🤖',",
    "    layout='wide',",
    "    initial_sidebar_state='expanded'",
    ")",
    "",
    "# Custom CSS",
    "st.markdown('''",
    "<style>",
    "    .main-header {",
    "        font-size: 2.5rem;",
    "        color: #1f77b4;",
    "        text-align: center;",
    "        margin-bottom: 2rem;",
    "    }",
    "    .emotion-badge {",
    "        display: inline-block;",
    "        padding: 0.25rem 0.5rem;",
    "        border-radius: 1rem;",
    "        font-size: 0.8rem;",
    "        font-weight: bold;",
    "        margin-left: 0.5rem;",
    "    }",
    "    .emotion-joy { background-color: #ffeb3b; color: #000; }",
    "    .emotion-sadness { background-color: #2196f3; color: white; }",
    "    .emotion-anger { background-color: #f44336; color: white; }",
    "    .emotion-fear { background-color: #9c27b0; color: white; }",
    "    .emotion-neutral { background-color: #9e9e9e; color: white; }",
    "    .disclaimer {",
    "        background-color: #cc9c00;",
    "        border: 1px solid #ffeaa7;",
    "        border-radius: 0.5rem;",
    "        padding: 1rem;",
    "        margin-top: 1rem;",
    "        font-size: 0.9rem;",
    "    }",
    "</style>",
    "''', unsafe_allow_html=True)",
    "",
    "# Initialize session state",
    "if 'messages' not in st.session_state:",
    "    st.session_state.messages = []",
    "if 'emotion_history' not in st.session_state:",
    "    st.session_state.emotion_history = []",
    "if 'model_loaded' not in st.session_state:",
    "    st.session_state.model_loaded = False",
    "",
    "class Phi2EmpathyAIChatbot:",
    "    def __init__(self):",
    "        self.emotion_classifier = None",
    "        self.sentence_model = None",
    "        self.language_model = None",
    "        self.tokenizer = None",
    "        self.faiss_index = None",
    "        self.response_data = []",
    "        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'",
    "        self.loading_status = 'Not started'",
    "        self.corpus_data = None",
    "        ",
    "    def load_models(self):",
    "        try:",
    "            st.info('🔄 Loading emotion detection model...')",
    "            self.emotion_classifier = pipeline(",
    "                'text-classification',",
    "                model='bhadresh-savani/distilbert-base-uncased-emotion',",
    "                return_all_scores=True,",
    "                device=0 if torch.cuda.is_available() else -1",
    "            )",
    "            ",
    "            st.info('🔄 Loading sentence transformer...')",
    "            self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')",
    "            ",
    "            st.info('🔄 Loading Phi-2 model...')",
    "            try:",
    "                model_name = 'microsoft/phi-2'",
    "                self.tokenizer = AutoTokenizer.from_pretrained(",
    "                    model_name,",
    "                    trust_remote_code=True",
    "                )",
    "                ",
    "                self.language_model = AutoModelForCausalLM.from_pretrained(",
    "                    model_name,",
    "                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,",
    "                    device_map='auto' if torch.cuda.is_available() else None,",
    "                    trust_remote_code=True",
    "                )",
    "                ",
    "                if self.tokenizer.pad_token is None:",
    "                    self.tokenizer.pad_token = self.tokenizer.eos_token",
    "                    ",
    "                st.success('✅ Phi-2 model loaded successfully!')",
    "                ",
    "            except Exception as e:",
    "                st.warning(f'⚠️ Phi-2 failed, using DistilGPT2: {e}')",
    "                self.tokenizer = AutoTokenizer.from_pretrained('distilgpt2')",
    "                self.language_model = AutoModelForCausalLM.from_pretrained('distilgpt2')",
    "                if self.tokenizer.pad_token is None:",
    "                    self.tokenizer.pad_token = self.tokenizer.eos_token",
    "                st.success('✅ DistilGPT2 model loaded!')",
    "            ",
    "            st.info('🔄 Loading corpus.json...')",
    "            self._load_existing_corpus()",
    "            ",
    "            st.info('🔄 Creating RAG system...')",
    "            self._create_rag_system()",
    "            ",
    "            return True",
    "            ",
    "        except Exception as e:",
    "            st.error(f'❌ Error loading models: {str(e)}')",
    "            return False",
    "    ",
    "    def _load_existing_corpus(self):",
    "        if os.path.exists('corpus.json'):",
    "            with open('corpus.json', 'r', encoding='utf-8') as f:",
    "                self.corpus_data = json.load(f)",
    "            total_responses = sum(len(responses) for responses in self.corpus_data.values())",
    "            st.success(f'✅ Loaded corpus with {total_responses} responses')",
    "        else:",
    "            st.error('❌ corpus.json not found')",
    "            self.corpus_data = None",
    "    ",
    "    def _create_rag_system(self):",
    "        if not self.corpus_data:",
    "            return",
    "        ",
    "        self.response_data = []",
    "        for emotion, responses in self.corpus_data.items():",
    "            for response in responses:",
    "                self.response_data.append({",
    "                    'text': f\"{response['situation']} {response['response_template']}\",",
    "                    'situation': response['situation'],",
    "                    'response_template': response['response_template'],",
    "                    'emotion': emotion,",
    "                    'original_emotion': response['emotion']",
    "                })",
    "        ",
    "        texts = [item['text'] for item in self.response_data]",
    "        embeddings = self.sentence_model.encode(texts)",
    "        ",
    "        dimension = embeddings.shape[1]",
    "        self.faiss_index = faiss.IndexFlatIP(dimension)",
    "        ",
    "        embeddings_normalized = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)",
    "        self.faiss_index.add(embeddings_normalized.astype('float32'))",
    "        ",
    "        st.success(f'✅ RAG system created with {self.faiss_index.ntotal} vectors')",
    "    ",
    "    def detect_emotion(self, text):",
    "        if not self.emotion_classifier:",
    "            return 'neutral', 0.5",
    "        ",
    "        try:",
    "            predictions = self.emotion_classifier(text)",
    "            top_prediction = max(predictions[0], key=lambda x: x['score'])",
    "            return top_prediction['label'], top_prediction['score']",
    "        except Exception as e:",
    "            return 'neutral', 0.5",
    "    ",
    "    def retrieve_relevant_responses(self, query, emotion_category, top_k=2):",
    "        if not self.faiss_index:",
    "            return []",
    "        ",
    "        query_embedding = self.sentence_model.encode([query])",
    "        query_embedding_normalized = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)",
    "        ",
    "        scores, indices = self.faiss_index.search(query_embedding_normalized.astype('float32'), top_k * 2)",
    "        ",
    "        candidates = []",
    "        for score, idx in zip(scores[0], indices[0]):",
    "            response = self.response_data[idx]",
    "            emotion_match = 1.0 if response['emotion'] == emotion_category else 0.0",
    "            combined_score = 0.7 * score + 0.3 * emotion_match",
    "            ",
    "            candidates.append({",
    "                'response': response,",
    "                'score': float(combined_score),",
    "                'emotion_match': emotion_match",
    "            })",
    "        ",
    "        candidates.sort(key=lambda x: x['score'], reverse=True)",
    "        return candidates[:top_k]",
    "    ",
    "    def generate_response(self, user_message):",
    "        if not self.language_model:",
    "            return 'I am not loaded yet. Please wait.', 'neutral', 0.5, []",
    "        ",
    "        try:",
    "            emotion, confidence = self.detect_emotion(user_message)",
    "            ",
    "            emotion_mapping = {",
    "                'joy': 'happiness', 'happiness': 'happiness', 'excitement': 'happiness',",
    "                'love': 'happiness', 'surprise': 'happiness',",
    "                'sadness': 'sadness', 'sorrow': 'sadness', 'grief': 'sadness',",
    "                'anger': 'anger', 'rage': 'anger', 'fury': 'anger',",
    "                'fear': 'fear',",
    "                'neutral': 'neutral', 'calm': 'neutral', 'peaceful': 'neutral'",
    "            }",
    "            emotion_category = emotion_mapping.get(emotion.lower(), 'neutral')",
    "            ",
    "            relevant_responses = self.retrieve_relevant_responses(user_message, emotion_category, top_k=2)",
    "            ",
    "            response = self._generate_with_phi2_rag(user_message, emotion, relevant_responses)",
    "            ",
    "            return response, emotion, confidence, relevant_responses",
    "            ",
    "        except Exception as e:",
    "            return 'I encountered an error. Please try again.', 'neutral', 0.5, []",
    "    ",
    "    def _generate_with_phi2_rag(self, user_message, emotion, relevant_responses):",
    "        try:",
    "            context_parts = [",
    "                'You are a helpful and empathetic AI assistant.',",
    "                f'The user is feeling {emotion}.',",
    "                '',",
    "                'Respond with empathy and understanding. Be direct and conversational.',",
    "                'Do not use any special tags, brackets, or formatting.',",
    "                '',",
    "                'Examples of good responses:',",
    "                ''",
    "            ]",
    "            ",
    "            for i, candidate in enumerate(relevant_responses, 1):",
    "                response_text = candidate['response']['response_template']",
    "                situation = candidate['response']['situation']",
    "                context_parts.append(f'Example {i} (situation: {situation}): {response_text}')",
    "            ",
    "            context_parts.extend([",
    "                '',",
    "                f'Now respond empathetically to this user message:',",
    "                f'User: {user_message}',",
    "                'You:'",
    "            ])",
    "            ",
    "            context = '\\n'.join(context_parts)",
    "            ",
    "            inputs = self.tokenizer.encode(context, return_tensors='pt', truncation=True, max_length=512)",
    "            inputs = inputs.to(self.device)",
    "            ",
    "            with torch.no_grad():",
    "                outputs = self.language_model.generate(",
    "                    inputs,",
    "                    max_length=inputs.shape[1] + 150,",
    "                    num_return_sequences=1,",
    "                    temperature=0.7,",
    "                    do_sample=True,",
    "                    top_p=0.9,",
    "                    top_k=50,",
    "                    pad_token_id=self.tokenizer.eos_token_id,",
    "                    eos_token_id=self.tokenizer.eos_token_id,",
    "                    no_repeat_ngram_size=2,",
    "                    attention_mask=torch.ones_like(inputs),",
    "                    repetition_penalty=1.1",
    "                )",
    "            ",
    "            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)",
    "            ",
    "            if 'You:' in full_response:",
    "                response = full_response.split('You:')[-1].strip()",
    "            elif 'Assistant:' in full_response:",
    "                response = full_response.split('Assistant:')[-1].strip()",
    "            else:",
    "                response = full_response[len(context):].strip()",
    "            ",
    "            # Clean up response - remove XML tags and unwanted content",
    "            response = response.split('\\n')[0].strip()",
    "            ",
    "            # Remove XML-like tags",
    "            response = re.sub(r'<[^>]+>', '', response)",
    "            response = re.sub(r'&[^;]+;', '', response)",
    "            ",
    "            # Remove common unwanted patterns",
    "            unwanted_patterns = [",
    "                r'<empathy>.*?</empathy>',",
    "                r'<emitivity>.*?</emitivity>',",
    "                r'<response>.*?</response>',",
    "                r'<answer>.*?</answer>',",
    "                r'<bot>.*?</bot>',",
    "                r'<assistant>.*?</assistant>',",
    "                r'^[^a-zA-Z]*',  # Remove leading non-alphabetic characters",
    "                r'\\[.*?\\]',  # Remove content in brackets",
    "                r'\\(.*?\\)',  # Remove content in parentheses that might be metadata",
    "            ]",
    "            ",
    "            for pattern in unwanted_patterns:",
    "                response = re.sub(pattern, '', response, flags=re.IGNORECASE | re.DOTALL)",
    "            ",
    "            # Clean up extra whitespace",
    "            response = ' '.join(response.split())",
    "            ",
    "            # Remove common prefixes that models sometimes add",
    "            prefixes_to_remove = [",
    "                'Response:', 'Answer:', 'Bot:', 'Assistant:', 'AI:', 'Chatbot:',",
    "                'Here is my response:', 'My response is:', 'I would say:',",
    "                'Based on the context:', 'Given the situation:'",
    "            ]",
    "            ",
    "            for prefix in prefixes_to_remove:",
    "                if response.lower().startswith(prefix.lower()):",
    "                    response = response[len(prefix):].strip()",
    "            ",
    "            # Ensure response starts with a capital letter",
    "            if response and response[0].islower():",
    "                response = response[0].upper() + response[1:]",
    "            ",
    "            # Final cleanup",
    "            response = response.strip()",
    "            ",
    "            if len(response) < 20:",
    "                response = self._get_fallback_response(emotion)",
    "            ",
    "            return response",
    "            ",
    "        except Exception as e:",
    "            return self._get_fallback_response(emotion)",
    "    ",
    "    def _get_fallback_response(self, emotion):",
    "        fallback_responses = {",
    "            'happiness': 'I can feel your joy! That is wonderful to hear.',",
    "            'sadness': 'I am sorry you are feeling this way. I am here to listen.',",
    "            'anger': 'I understand you are frustrated. It is okay to feel angry.',",
    "            'fear': 'I understand you are anxious. It is normal to feel nervous.',",
    "            'neutral': 'I am here to listen and support you.'",
    "        }",
    "        return fallback_responses.get(emotion.lower(), 'I understand you are feeling this way.')",
    "",
    "def get_emotion_emoji(emotion):",
    "    emoji_map = {",
    "        'joy': '', 'happiness': '😊', 'excitement': '',",
    "        'love': '❤️', 'surprise': '',",
    "        'sadness': '😢', 'sorrow': '😢', 'grief': '',",
    "        'anger': '', 'rage': '', 'fury': '',",
    "        'fear': '', 'anxiety': '',",
    "        'neutral': '😐', 'calm': '', 'peaceful': '😌'",
    "    }",
    "    return emoji_map.get(emotion.lower(), '🤖')",
    "",
    "def get_emotion_class(emotion):",
    "    emotion_map = {",
    "        'joy': 'emotion-joy', 'happiness': 'emotion-joy', 'excitement': 'emotion-joy',",
    "        'love': 'emotion-joy', 'surprise': 'emotion-joy',",
    "        'sadness': 'emotion-sadness', 'sorrow': 'emotion-sadness', 'grief': 'emotion-sadness',",
    "        'anger': 'emotion-anger', 'rage': 'emotion-anger', 'fury': 'emotion-anger',",
    "        'fear': 'emotion-fear', 'anxiety': 'emotion-fear',",
    "        'neutral': 'emotion-neutral', 'calm': 'emotion-neutral', 'peaceful': 'emotion-neutral'",
    "    }",
    "    return emotion_map.get(emotion.lower(), 'emotion-neutral')",
    "",
    "def main():",
    "    st.markdown('<h1 class=\"main-header\"> Empathy AI Chatbot - Phi-2 + RAG</h1>', unsafe_allow_html=True)",
    "    st.markdown('---')",
    "    ",
    "    with st.sidebar:",
    "        st.header('🤖 Chatbot Status')",
    "        ",
    "        if not st.session_state.model_loaded:",
    "            if st.button('🚀 Load AI Models', type='primary'):",
    "                with st.spinner('Loading models...'):",
    "                    chatbot = Phi2EmpathyAIChatbot()",
    "                    if chatbot.load_models():",
    "                        st.session_state.chatbot = chatbot",
    "                        st.session_state.model_loaded = True",
    "                        st.success('✅ Models loaded successfully!')",
    "                        st.rerun()",
    "                    else:",
    "                        st.error('❌ Failed to load models')",
    "        ",
    "        if st.session_state.model_loaded:",
    "            st.success('✅ AI Models Ready')",
    "            st.info('💡 The chatbot is ready to chat!')",
    "            ",
    "            if st.button('🗑️ Clear Chat History'):",
    "                st.session_state.messages = []",
    "                st.session_state.emotion_history = []",
    "                st.rerun()",
    "        ",
    "        st.markdown('---')",
    "        st.header(' Emotion Analysis')",
    "        if st.session_state.emotion_history:",
    "            recent_emotions = st.session_state.emotion_history[-5:]",
    "            for i, (emotion, confidence) in enumerate(reversed(recent_emotions)):",
    "                emoji = get_emotion_emoji(emotion)",
    "                st.write(f'{emoji} {emotion} ({confidence:.2f})')",
    "        ",
    "        st.markdown('---')",
    "        st.markdown('''",
    "        ### ℹ️ About",
    "        This AI chatbot uses **Phi-2** and **RAG** with empathetic dialogues dataset.",
    "        ",
    "        **Features:**",
    "        - Phi-2 language model",
    "        - RAG system",
    "        - Emotion detection",
    "        - Context-aware responses",
    "        - Clean response generation",
    "        ''')",
    "    ",
    "    if not st.session_state.model_loaded:",
    "        st.info(' Please load the AI models using the sidebar button to start chatting.')",
    "        ",
    "        st.markdown('### 💬 Example Messages to Try:')",
    "        example_messages = [",
    "            'I am so excited about my new job!',",
    "            'I can not believe my best friend moved away.',",
    "            'This traffic is making me furious!',",
    "            'I am terrified of the presentation tomorrow',",
    "            'I had a peaceful walk in the park today.'",
    "        ]",
    "        ",
    "        for msg in example_messages:",
    "            st.write(f'• {msg}')",
    "    ",
    "    else:",
    "        st.markdown('### 💬 Chat with Empathy AI')",
    "        ",
    "        for message in st.session_state.messages:",
    "            with st.chat_message(message['role']):",
    "                st.markdown(message['content'])",
    "                if message['role'] == 'assistant' and 'emotion' in message:",
    "                    emotion = message['emotion']",
    "                    confidence = message['confidence']",
    "                    emoji = get_emotion_emoji(emotion)",
    "                    st.markdown(f'<span class=\"emotion-badge {get_emotion_class(emotion)}\">{emoji} {emotion} ({confidence:.2f})</span>', unsafe_allow_html=True)",
    "                    ",
    "                    if 'rag_responses' in message and message['rag_responses']:",
    "                        with st.expander('🔍 RAG Context from Dataset'):",
    "                            for i, rag_resp in enumerate(message['rag_responses'], 1):",
    "                                st.write(f'**Context {i}:** {rag_resp[\"response\"][\"response_template\"]}')",
    "                                st.write(f'**Situation:** {rag_resp[\"response\"][\"situation\"]}')",
    "                                st.write(f'**Score:** {rag_resp[\"score\"]:.3f}')",
    "        ",
    "        if prompt := st.chat_input('Type your message here...'):",
    "            st.session_state.messages.append({'role': 'user', 'content': prompt})",
    "            ",
    "            with st.chat_message('user'):",
    "                st.markdown(prompt)",
    "            ",
    "            with st.chat_message('assistant'):",
    "                with st.spinner('Thinking with Phi-2 and RAG...'):",
    "                    response, emotion, confidence, rag_responses = st.session_state.chatbot.generate_response(prompt)",
    "                    st.markdown(response)",
    "                    ",
    "                    emoji = get_emotion_emoji(emotion)",
    "                    st.markdown(f'<span class=\"emotion-badge {get_emotion_class(emotion)}\">{emoji} {emotion} ({confidence:.2f})</span>', unsafe_allow_html=True)",
    "                    ",
    "                    st.session_state.emotion_history.append((emotion, confidence))",
    "            ",
    "            st.session_state.messages.append({",
    "                'role': 'assistant', ",
    "                'content': response,",
    "                'emotion': emotion,",
    "                'confidence': confidence,",
    "                'rag_responses': rag_responses",
    "            })",
    "        ",
    "        st.markdown('''",
    "        <div class=\"disclaimer\">",
    "            <strong>⚠️ Disclaimer:</strong> I am an AI assistant designed to provide empathetic responses. ",
    "            I am not a substitute for professional mental health care. If you are experiencing severe ",
    "            emotional distress, please consider reaching out to a mental health professional.",
    "        </div>",
    "        ''', unsafe_allow_html=True)",
    "",
    "if __name__ == '__main__':",
    "    main()"
]

# Write the file line by line
with open('app.py', 'w') as f:
    for line in lines:
        f.write(line + '\n')

print("✅ Fixed app file created with response cleaning built-in!")

✅ Fixed app file created with response cleaning built-in!


# Run streamlit

In [22]:
ngrok.set_auth_token("2HRoVk7gLJyyeljEiNCWNIF5JM1_4SKHRQkgChtBCSZPV7NmY")  # Replace with your actual auth token

# Kill any existing ngrok tunnels
ngrok.kill()

# Start the Streamlit app in the background
print("Starting Streamlit app...")
# Use nohup and & to keep the process running in the background
!nohup streamlit run app.py &

# Give Streamlit a moment to start
time.sleep(10) # Increased sleep time to ensure Streamlit starts

# Create public tunnel
try:
    public_url = ngrok.connect(addr="8501", proto="http")
    print(f"🌐 Public URL: {public_url}")
    print("📱 Share this URL to access your chatbot from anywhere!")

    # Keep the tunnel open until interrupted
    print("Press Ctrl+C to stop ngrok tunnel.")
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Shutting down ngrok tunnel...")
    ngrok.kill()
except Exception as e:
    print(f"\n❌ Error setting up ngrok tunnel: {e}")
    print("Please check if Streamlit is running correctly on port 8501.")

Starting Streamlit app...
nohup: appending output to 'nohup.out'


KeyboardInterrupt: 